In [1]:
!pip install -q upgini catboost pandas scikit-learn

import pandas as pd
from catboost import CatBoostRegressor
from catboost.utils import eval_metric

from upgini import FeaturesEnricher, SearchKey
from upgini.metadata import CVType


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.2/51.2 kB 456.4 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 309.5/309.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.3/65.3 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.9/85.9 kB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 60.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.8/139.8 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 153.3/153.3 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 125.0/125.0 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 470.7/470.7 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 19.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3

In [5]:
DATA_URL = "https://raw.githubusercontent.com/kadiravyshnavi/salesforecast-dataset/main/train.csv.zip"

df = pd.read_csv(DATA_URL)
df = df.sample(n=900, random_state=0)

df["store"] = df["store"].astype(str)
df["item"] = df["item"].astype(str)
df["date"] = pd.to_datetime(df["date"])

df = df.sort_values("date").reset_index(drop=True)


In [6]:
train = df[df["date"] < "2017-01-01"]
test = df[df["date"] >= "2017-01-01"]

X_train = train.drop("sales", axis=1)
y_train = train["sales"]

X_test = test.drop("sales", axis=1)
y_test = test["sales"]


In [8]:
enricher = FeaturesEnricher(
    search_keys={"date": SearchKey.DATE},
    cv=CVType.time_series
)

enricher.fit(X_train, y_train, eval_set=[(X_test, y_test)])
X_train_enriched = enricher.transform(X_train, y_train)
X_test_enriched = enricher.transform(X_test, y_test)


upgini_model = CatBoostRegressor(
    verbose=False,
    random_state=0,
    allow_writing_files=False
)

upgini_model.fit(X_train_enriched, y_train)
upgini_preds = upgini_model.predict(X_test_enriched)

upgini_smape = eval_metric(
    y_test.values,
    upgini_preds,
    "SMAPE"
)

upgini_smape


[============================================================] 100% Finished

WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history

Detected task type: ModelTaskType.REGRESSION. Reason: date search key is present, treating as regression
You can set task type manually with argument `model_task_type` of FeaturesEnricher constructor if task type detected incorrectly

WARNING #2: Your training sample is unstable in number of rows per date. It is recommended to redesign the training sample



Column name,Status,Errors
target,All valid,-
date,All valid,-




Running search request, search_id=291e0619-e88f-44f5-b236-af6827aac7d3
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com



f_autofe_roll_2d_min_4199c37ccb,0.6658,100.0000,"7.0, 85.0, 53.0",Training dataset,AutoFE: features from Training dataset,
f_autofe_roll_3d_median_97804b26e6,0.0465,100.0000,"52.5, 86.0, 123.0",Training dataset,AutoFE: features from Training dataset,


...

[============================================================] 100% Finished

You use Trial access to Upgini data enrichment. Limit for Trial: 1000 rows. You have already enriched: 717 rows.
WARNING #1: Search started with DATE search key only
Try to add other keys like the COUNTRY, POSTAL_CODE, PHONE NUMBER, EMAIL/HEM, IP to your training dataset
for search through all the available data sources.
See docs https://github.com/upgini/upgini#-total-239-countries-and-up-to-41-years-of-history



Column name,Status,Errors
date,All valid,-




Running transform request, id=05b069c6-8621-4853-8ed5-77ac05bc4721
We'll send email notification once it's completed, just use your personal api_key from profile.upgini.com

Retrieving selected features from data sources...


[26.788911417095573]

In [9]:
enricher.feature_names_


['f_autofe_roll_2d_min_4199c37ccb', 'f_autofe_roll_3d_median_97804b26e6']